# Juego del gato

## Librerías

In [1]:
import numpy as np #librería para manipular arreglos de números y hacer operaciones sobre ellos
import pandas as pd #librería para manejar "bases de datos" en python (el pan de cada día para 100cia de datos)
import matplotlib.pyplot as plt #librearía para graficar
import seaborn as sns #librería para graficar un poco más alto que matplotlib

In [2]:
#explicar np.isin y filtros de arrays a[b]

In [3]:
[1, 5, 7, 9, 0]

[1, 5, 7, 9, 0]

In [4]:
class Gato():
    "Juego del gato"
    def __init__(self):
        self.used_positions = []
        self.won = False
    
    def return_available_actions(self):
        available_filter = ~np.isin(range(1,10), self.used_positions)
        available = np.array(range(1,10))[available_filter]
        return available
    
    def someone_won(self, positions):
        won = False
        winning_combinations = [[1,2,3], [4,5,6], [7,8,9], 
                                [1,4,7], [2,5,8], [3,6,9],
                                [1,5,9], [7,5,3]
                               ]
        for combination in winning_combinations:
            if np.isin(positions, combination).sum() == 3:
                won = True
        self.won = won
        return won
    
        
    
    def is_over(self):
        if self.won:
            return True
        else:
            return len(self.used_positions)==9
            
    def reset(self):
        self.used_positions = []

class Player():
    def __init__(self, env, debug = False):
        
        self.env = env
        self.current_state = [0,0,0,0,0]
        self.observations = []
        self.actions = []
        self.rewards = []
        
        self.debug = debug
    
    def take_step(self):
        available_positions = self.env.return_available_actions()
        if self.debug:
            print(f'Available: {available_positions}')
        #########policy###########
        action = np.random.choice(available_positions)
        if self.debug:
            print(f'Action chosen: {action}')
        ##########################
        self.observations.append(list(np.copy(self.current_state)))
        self.current_state[len(self.actions)] = action
        self.actions.append(action)
        
        self.env.used_positions.append(action)
        reward = self.env.someone_won(self.current_state)
        self.rewards.append(reward)
        if reward:
            if self.debug:
                print('Won!')
        pass

In [19]:
###chatGPT optimization

class Gato():
    "Juego del gato"
    def __init__(self):
        self.used_positions = []
    
    def return_available_actions(self):
        available_filter = ~np.isin(range(1, 10), self.used_positions)
        available = np.array(range(1, 10))[available_filter]
        return available
    
    def someone_won(self, positions):
        winning_combinations = [[1, 2, 3], [4, 5, 6], [7, 8, 9],
                                [1, 4, 7], [2, 5, 8], [3, 6, 9],
                                [1, 5, 9], [7, 5, 3]
                               ]
        for combination in winning_combinations:
            if np.isin(positions, combination).sum() == 3:
                return True
        return False
    
    def is_over(self):
        return self.someone_won(self.used_positions) or len(self.used_positions) == 9

    
class Player():
    def __init__(self, env, debug=False):
        self.env = env
        self.current_state = [0, 0, 0, 0, 0]
        self.actions = []
        self.rewards = []
        self.debug = debug
    
    def take_step(self):
        available_positions = self.env.return_available_actions()
        if self.debug:
            print(f'Available: {available_positions}')
        action = np.random.choice(available_positions)
        if self.debug:
            print(f'Action chosen: {action}')
        self.current_state[len(self.actions)] = action
        self.actions.append(action)
        self.env.used_positions.append(action)
        reward = self.env.someone_won(self.current_state)
        self.rewards.append(reward)
        if reward and self.debug:
            print('Won!')


In [8]:
runs = []
run_id = 0
for i in range(10000):
    gato = Gato()
    player_1 = Player(gato, debug=False)
    player_2 = Player(gato, debug=False)
    players = [(player_1,1), (player_2,2)]
    won = gato.won
    turn = 0
    i=0
    while not gato.is_over():
        player = players[turn][0]
        #print(f'Player {players[turn][1]}')
        player.take_step()
        turn += (-1)**i
        i +=1
    X_1 = pd.DataFrame(np.array(player_1.observations)[:,:-1], columns=[f'step_{i}' for i in range(1,5)])
    y_1 = pd.DataFrame(np.array(player_1.actions).reshape(-1,1), columns = ['action'])
    rewards_1 = pd.DataFrame(np.array(player_1.rewards).astype(int).reshape(-1,1), columns=['rewards'])
    agent_1_id = pd.DataFrame(np.array([1]*len(player_1.actions)).astype(int).reshape(-1,1), columns=['agent_id'])
    data_1 = pd.concat((agent_1_id,X_1,y_1,rewards_1), axis = 1)
    X_2 = pd.DataFrame(np.array(player_2.observations)[:,:-1], columns=[f'step_{i}' for i in range(1,5)])
    y_2 = pd.DataFrame(np.array(player_2.actions).reshape(-1,1), columns = ['action'])
    rewards_2 = pd.DataFrame(np.array(player_2.rewards).astype(int).reshape(-1,1), columns=['rewards'])
    agent_2_id = pd.DataFrame(np.array([2]*len(player_2.actions)).astype(int).reshape(-1,1), columns=['agent_id'])
    data_2 = pd.concat((agent_2_id,X_2,y_2,rewards_2), axis = 1)
    run_data = pd.concat((data_1, data_2), ignore_index=True)
    run_ids = pd.DataFrame(np.array([run_id]*len(run_data)).astype(int).reshape(-1,1), columns=['run_id'])
    run_data = pd.concat((run_ids,run_data), axis=1)
    runs.append(run_data)
    run_id += 1

In [9]:
data = pd.concat(runs, ignore_index=True)
data.loc[:,['id']] = data.run_id.astype(str)+'_'+data.agent_id.astype(str)
data.head()

,run_id,agent_id,step_1,step_2,step_3,step_4,action,rewards,id
0,0,1,0,0,0,0,2,0,0_1
1,0,1,2,0,0,0,5,0,0_1
2,0,1,2,5,0,0,1,0,0_1
3,0,1,2,5,1,0,7,0,0_1
4,0,1,2,5,1,7,3,1,0_1


In [10]:
#en el límite, la proporción es 66-33
data.loc[data.rewards==1,['run_id','agent_id']].drop_duplicates()\
    .agent_id.value_counts(normalize=True).round(2)

agent_id
1    0.67
2    0.33
Name: proportion, dtype: float64

In [11]:
winning_ids = data.loc[data.rewards==1,['id']].id.unique()
data = data.loc[data.id.isin(winning_ids)]
data.head()

,run_id,agent_id,step_1,step_2,step_3,step_4,action,rewards,id
0,0,1,0,0,0,0,2,0,0_1
1,0,1,2,0,0,0,5,0,0_1
2,0,1,2,5,0,0,1,0,0_1
3,0,1,2,5,1,0,7,0,0_1
4,0,1,2,5,1,7,3,1,0_1


In [12]:
X = data.loc[:,['step_1','step_2','step_3','step_4']]
y = data.action

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
model = GradientBoostingClassifier()

In [15]:
model.fit(X,y)

GradientBoostingClassifier()

In [16]:
model.feature_importances_

array([0.37887006, 0.3415805 , 0.21688985, 0.06265959])

In [18]:
test = pd.DataFrame([[1,0,0,0]], columns=X.columns)
model.predict_proba(test).round(2)

array([[0.01, 0.11, 0.13, 0.12, 0.15, 0.1 , 0.13, 0.1 , 0.15]])

$$
\begin{pmatrix}
1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 
\end{pmatrix}
$$
